## <font color='#f7f'> [0] Initialization </font>

In [ ]:
# remove residues from notebook
!rm -rf sample_data

# get the zip containing the cat images
!wget https://github.com/Crash285github/nyaural_catwork/raw/main/data/cats.zip -P data/

# extract the cats into data/
from zipfile import ZipFile
with ZipFile('data/cats.zip', 'r') as cats:
  cats.extractall('data')

# remove the downloaded zip
!rm data/cats.zip

# <font color='#f77'>[1] Data manipulation </font>

#### <font color='#777'> [1.0] Imports</font>

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
import os
import matplotlib.pyplot as plt
%matplotlib inline

#### <font color='#47f'> [1.1] Dataset class</font> 

In [ ]:
class CatDataset(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        self.total_imgs = os.listdir(main_dir)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc)
        
        # we permute: (3, 64, 64) --> (64, 64, 3), this way matplotlib can show it
        tensor_image = self.transform(image).permute(1,2,0)
        return tensor_image

#### <font color='#47f'>[1.2] Create dataset</font>

In [ ]:
dataset = CatDataset('data', transforms.ToTensor())

plt.imshow(dataset[10])
print(dataset[10])